In [ ]:
!pip install -r "D:\hang_test\04.01\kcbert\ratsnlp\requirements.txt"

In [3]:
%cd "C:\Users\coms\Desktop\04.07\kcbert\ratsnlp\ratsnlp\nlpbook"

C:\Users\coms\Desktop\04.07\kcbert\ratsnlp\ratsnlp\nlpbook


In [4]:
import torch
from classification import ClassificationTrainArguments
args = ClassificationTrainArguments(
    pretrained_model_name="beomi/kcbert-base",  #프리트레인 마친 언어모델의 이름
    downstream_corpus_name="data", # 다운스트림 데이터의 이름
    downstream_model_dir="C:/Users/coms/Desktop/04.07/kcbert/result",  # 파인튜닝된 모델의 체크포인트가 저장될 위치. 
    downstream_corpus_root_dir =  "C:/Users/coms/Desktop/04.07/kcbert/result/Korpora",
    batch_size=32 if torch.cuda.is_available() else 4, #TPU로 설정했다면 4
    learning_rate=5e-5,   
    max_seq_length=128,  # 패딩과 같은 의미 / 이보다 긴문장은 자르고 짧은 문장은 padding을 진행한다. 
    epochs=1,
    tpu_cores=0 if torch.cuda.is_available() else 8, # GPU면 TPU 코어수 0 TPU면 8
    seed=7,
)

In [17]:
args.downstream_corpus_root_dir

'C:/Users/coms/Desktop/04.07/kcbert/result/Korpora'

In [18]:
from utils import set_seed
set_seed(args)

set seed: 7


In [19]:
from utils import set_logger
set_logger(args)

INFO:ratsnlp:Training/evaluation parameters ClassificationTrainArguments(pretrained_model_name='beomi/kcbert-base', downstream_task_name='document-classification', downstream_corpus_name='data', downstream_corpus_root_dir='C:/Users/coms/Desktop/04.07/kcbert/result/Korpora', downstream_model_dir='C:/Users/coms/Desktop/04.07/kcbert/result', max_seq_length=128, save_top_k=1, monitor='min val_loss', seed=7, overwrite_cache=False, force_download=False, test_mode=False, learning_rate=5e-05, epochs=1, batch_size=32, cpu_workers=16, fp16=False, tpu_cores=0)
INFO:ratsnlp:Training/evaluation parameters ClassificationTrainArguments(pretrained_model_name='beomi/kcbert-base', downstream_task_name='document-classification', downstream_corpus_name='data', downstream_corpus_root_dir='C:/Users/coms/Desktop/04.07/kcbert/result/Korpora', downstream_model_dir='C:/Users/coms/Desktop/04.07/kcbert/result', max_seq_length=128, save_top_k=1, monitor='min val_loss', seed=7, overwrite_cache=False, force_download

In [ ]:
from Korpora import Korpora
Korpora.fetch(
    corpus_name=args.downstream_corpus_name,
    root_dir= args.downstream_corpus_root_dir,
    force_download=True,
)

In [20]:
from transformers import BertTokenizer
tokenizer = BertTokenizer.from_pretrained(
    args.pretrained_model_name,
    do_lower_case=False,
)

In [21]:
from torch.utils.data import DataLoader, SequentialSampler, RandomSampler
from classification import NsmcCorpus, ClassificationDataset
from data_utils import data_collator
corpus = NsmcCorpus()    #“문장(영화 리뷰) + 레이블(긍정, 부정)”

## 데이터 토크나이저 하기 : 모델이 학습할 수 있는 형태로 가공
train_dataset = ClassificationDataset(  
    args=args,
    corpus=corpus,#“문장(영화 리뷰) + 레이블(긍정, 부정)”
    tokenizer=tokenizer, #모델이 학습할 수 있는 형태로 가공
    mode="train",
)

# 전체 인스턴스 가운데 배치크기 만큼 뽑아 배치형태로 가공하기 
train_dataloader = DataLoader(
    train_dataset,
    batch_size=args.batch_size,
    sampler=RandomSampler(train_dataset, replacement=False), # 배치를 만들 대 전체 인스턴스 중 배치사이즈 수 만큼 비복원 추출을 한다. 
    collate_fn=data_collator, # 리스트 형의 ClassificationDataset를 tensor형으로 바꾼다. 
    drop_last=False,
    num_workers=args.cpu_workers,
)

INFO:ratsnlp:Creating features from dataset file at C:/Users/coms/Desktop/04.07/kcbert/result/Korpora\data
INFO:ratsnlp:Creating features from dataset file at C:/Users/coms/Desktop/04.07/kcbert/result/Korpora\data
INFO:ratsnlp:loading train data... LOOKING AT C:/Users/coms/Desktop/04.07/kcbert/result/Korpora\data\ratings_train.txt
INFO:ratsnlp:loading train data... LOOKING AT C:/Users/coms/Desktop/04.07/kcbert/result/Korpora\data\ratings_train.txt
INFO:ratsnlp:tokenize sentences, it could take a lot of time...
INFO:ratsnlp:tokenize sentences, it could take a lot of time...


ClassificationExample(text_a='카카오게임즈 첫 MMORPG 테라 클래식 내달 ', text_b=None, label='0')


INFO:ratsnlp:tokenize sentences [took 2.695 s]
INFO:ratsnlp:tokenize sentences [took 2.695 s]
INFO:ratsnlp:*** Example ***
INFO:ratsnlp:*** Example ***
INFO:ratsnlp:sentence: 카카오게임즈 첫 MMORPG 테라 클래식 내달 
INFO:ratsnlp:sentence: 카카오게임즈 첫 MMORPG 테라 클래식 내달 
INFO:ratsnlp:tokens: [CLS] 카카오 ##게임 ##즈 첫 M ##M ##O ##R ##P ##G 테 ##라 클 ##래 ##식 내 ##달 [SEP] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD]
IN

In [22]:
from data_utils import data_collator
val_dataset = ClassificationDataset(
    args=args,
    corpus=corpus,
    tokenizer=tokenizer,
    mode="test",
)
val_dataloader = DataLoader(
    val_dataset,
    batch_size=args.batch_size,
    sampler=SequentialSampler(val_dataset), # 평가용 데이터 로더 구축. batch_size만큼의 갯수만큼을 인스턴스 순서대로 추출하는 역할을 합니다. #학습은 랜덤 평가는 순서대로
    collate_fn=data_collator,
    drop_last=False,
    num_workers=args.cpu_workers,
)

INFO:ratsnlp:Creating features from dataset file at C:/Users/coms/Desktop/04.07/kcbert/result/Korpora\data
INFO:ratsnlp:Creating features from dataset file at C:/Users/coms/Desktop/04.07/kcbert/result/Korpora\data
INFO:ratsnlp:loading test data... LOOKING AT C:/Users/coms/Desktop/04.07/kcbert/result/Korpora\data\ratings_test.txt
INFO:ratsnlp:loading test data... LOOKING AT C:/Users/coms/Desktop/04.07/kcbert/result/Korpora\data\ratings_test.txt
INFO:ratsnlp:tokenize sentences, it could take a lot of time...
INFO:ratsnlp:tokenize sentences, it could take a lot of time...


ClassificationExample(text_a='불법 보조금에 공짜 마케팅까지…이통3사 갤S8 진흙탕 ', text_b=None, label='2')


INFO:ratsnlp:tokenize sentences [took 1.078 s]
INFO:ratsnlp:tokenize sentences [took 1.078 s]
INFO:ratsnlp:*** Example ***
INFO:ratsnlp:*** Example ***
INFO:ratsnlp:sentence: 불법 보조금에 공짜 마케팅까지…이통3사 갤S8 진흙탕 
INFO:ratsnlp:sentence: 불법 보조금에 공짜 마케팅까지…이통3사 갤S8 진흙탕 
INFO:ratsnlp:tokens: [CLS] 불법 보조금 ##에 공짜 마케팅 ##까지 [UNK] 이 ##통 ##3 ##사 갤 ##S ##8 진 ##흙 ##탕 [SEP] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [P

In [23]:
# 모델 초기화
from transformers import BertConfig, BertForSequenceClassification
pretrained_model_config = BertConfig.from_pretrained(
    args.pretrained_model_name,
    num_labels=corpus.num_labels,
)

In [24]:
model = BertForSequenceClassification.from_pretrained(
        args.pretrained_model_name,
        config=pretrained_model_config,
)

Some weights of the model checkpoint at beomi/kcbert-base were not used when initializing BertForSequenceClassification: ['cls.seq_relationship.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.decoder.weight', 'cls.seq_relationship.weight', 'cls.predictions.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.decoder.bias', 'cls.predictions.transform.dense.weight']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initiali

In [25]:
from classification import ClassificationTask
task = ClassificationTask(model, args)

In [26]:
from trainer import get_trainer
trainer = get_trainer(args)

GPU available: True, used: True
TPU available: False, using: 0 TPU cores


In [29]:
from trainer import get_trainer

trainer.fit(
    task,
    train_dataloader=train_dataloader,
    val_dataloaders=val_dataloader,
)

LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]

  | Name  | Type                          | Params
--------------------------------------------------------
0 | model | BertForSequenceClassification | 108 M 
--------------------------------------------------------
108 M     Trainable params
0         Non-trainable params
108 M     Total params
435.696   Total estimated model params size (MB)


Epoch 0:   0%|          | 0/1213 [00:00<?, ?it/s] 

In [28]:
import gc
gc.collect()
torch.cuda.empty_cache()

In [12]:
# 토크나이저 초기화하기
from transformers import BertTokenizer
tokenizer = BertTokenizer.from_pretrained(
    args.pretrained_model_name,
    do_lower_case=False,
)

In [9]:
from classification import ClassificationDeployArguments
from transformers import BertConfig, BertForSequenceClassification
args = ClassificationDeployArguments(
    pretrained_model_name="beomi/kcbert-base",  
    downstream_model_dir="C:/Users/coms/Desktop/04.07/kcbert/result", 
    max_seq_length=128,  
)



fine_tuned_model_ckpt = torch.load(   
    args.downstream_model_checkpoint_fpath,
    map_location=torch.device("cpu")
)
pretrained_model_config = BertConfig.from_pretrained(
    args.pretrained_model_name,
    num_labels=fine_tuned_model_ckpt['state_dict']['model.classifier.bias'].shape.numel(),
)

model = BertForSequenceClassification(pretrained_model_config)


model.load_state_dict({k.replace("model.", ""): v for k, v in fine_tuned_model_ckpt['state_dict'].items()})

model.eval()

downstream_model_checkpoint_fpath: C:/Users/coms/Desktop/04.07/kcbert/result\epoch=0-val_loss=0.46.ckpt


BertForSequenceClassification(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(30000, 768, padding_idx=0)
      (position_embeddings): Embedding(300, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0): BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e-12, element

In [10]:
def inference_fn(sentence):
    inputs = tokenizer(
        [sentence],
        max_length=args.max_seq_length,
        padding="max_length",
        truncation=True,
    )
    with torch.no_grad():
        outputs = model(**{k: torch.tensor(v) for k, v in inputs.items()})  #inputs를 파이토치 텐서로 바꾼후 모델 계산하기
        prob = outputs.logits.softmax(dim=1)  ## outputs.logit은 soft-max함수에 넣기 전 로짓(logit)형태이다. 
        zero_prob = round(prob[0][0].item(), 4) ## 긍정/부정일 확률을 소수점 4자리로 반올림
        one_prob = round(prob[0][1].item(), 4)
        two_prob = round(prob[0][2].item(), 4)
        three_prob = round(prob[0][3].item(), 4)
        four_prob = round(prob[0][4].item(), 4)
        five_prob = round(prob[0][5].item(), 4)
        six_prob = round(prob[0][6].item(), 4)
    return prob

In [15]:
trial_text = pd.read_csv

tensor([[0.0200, 0.8924, 0.0665, 0.0072, 0.0078, 0.0039, 0.0023]])